In [1]:
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error as mse
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from math import sqrt
from math import log10
import math
import random
from sklearn.metrics import r2_score
import optuna
from optuna.trial import TrialState
import tkinter
from tkinter import filedialog
import os
warnings.simplefilter(action='ignore', category=FutureWarning)
import glob

#from matplotlib.figure import Figure
#from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
#NavigationToolbar2Tk)
%matplotlib qt
from tkinter import *
from tkinter import ttk
import ast

In [2]:
def browsefunc():
    filename = filedialog.askopenfilename()
    text.delete(0,END)
    text.insert(0,filename)

In [3]:
def plot_inputs(*args):
    filename = var.get()
    dense = float(ro.get())
    new_df = asad_plotting(filename,dense)[0]
    columns_values = ['SWOB','TFLO','SPPA','DHAP']
    fig, ax = plt.subplots(len(columns_values),figsize = (30,20))
    for i,column in enumerate(new_df[columns_values]):
          new_df[column].plot(kind = "line",ax = ax[i]).set_title(column)
    fig.tight_layout()
    fig.show()

In [4]:
def C_conc(ROP):
    return 0.01778*ROP + 0.505

In [5]:
def V_cut(C_conc,D_hole,D_dp,ROP):
    return ((ROP)/(36*(1-(D_dp/D_hole)**2)*C_conc))

In [6]:
def V_critical(V_cut,ESV):
    return V_cut + ESV

In [7]:
def visc_app(Yp,D_hole,D_dp,visc_p,V_critical):
    return visc_p + (5*Yp*(D_hole-D_dp))/V_critical

In [8]:
def ESV_true(visc_ap):
    if visc_ap < 53:
        return 0.00516*visc_ap + 3.006
    else:
        return 0.02554*(visc_ap - 53) + 3.28

In [9]:
def C_ang(angle):
    return 0.0342*angle - 0.000233*angle**2 - 0.213

In [10]:
def C_size(D50):
    return -1.04*D50 + 1.286

In [11]:
def C_mw(density):
    if density >=8.7:
        return 1 - 0.0333*(density - 8.7)
    else:
        return 1

In [12]:
def ESV_gen(C_an,C_s,C_m,ESV_tr):
    return ESV_tr*C_an*C_s*C_m

In [13]:
def flow_rate(D_hole,D_dp,ESV_gen):
    area = 3.14/4*(D_hole**2 - D_dp**2)*0.00694444
    return ESV_gen*area*448

In [14]:
def hampel_filter_pandas(input_series, window_size, n_sigmas=3):

    k = 1.4826 # scale factor for Gaussian distribution
    new_series = input_series.copy()

    # helper lambda function 
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    
    rolling_median = input_series.rolling(window=2*window_size, center=True).median()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = np.abs(input_series - rolling_median)

    indices = list(np.argwhere(diff > (n_sigmas * rolling_mad)).flatten())
    new_series[indices] = rolling_median[indices]
    
    return new_series

In [15]:
def asad_plotting(filename,density):
    df = pd.read_csv(filename,delim_whitespace=True)
    #df = pd.read_csv(filename)
    df = df.replace(-999.25,np.nan)
    df = df.dropna()
    depth = df['DEPT'][df.tail(1).index[0]]*3.28
    #annulus_loss = abs(df['DHAP'][df.tail(1).index[0]] - 0.052*depth*density)
    column_values = ['DEPT','SWOB','TFLO','SPPA','DHAP']
    df = df[column_values]
    df['TFLO'] = df['TFLO']
    df['SPPA'] = df['SPPA']
    df['SWOB'] = df['SWOB']
    df['DHAP'] = df['DHAP']
    df = df[df['SWOB']>0]
    df = df.reset_index(drop = True)
    
    result1 = hampel_filter_pandas(df['SWOB'], 10)
    result2 = hampel_filter_pandas(df['TFLO'], 10)
    result3 = hampel_filter_pandas(df['SPPA'], 10)
    result4 = hampel_filter_pandas(df['DHAP'], 10)
    new_df = pd.DataFrame({'DEPTH':df['DEPT'],'SWOB':result1,'TFLO':result2,'SPPA':result3,'DHAP':result4})
    return new_df,depth

In [16]:
def inputs_nn(new_df):
    #filename = var.get()
    #dense = float(ro.get())
    #new_df = asad_plotting(filename,['SWOB','TFLO','SPPA'],dense)[0]
    X = new_df[['SWOB','TFLO']]
    y = new_df['SPPA']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    sc_x = StandardScaler()
    X_tr_sc = sc_x.fit_transform(X_train)
    X_ts_sc = sc_x.transform(X_test)

    sc_y = StandardScaler()
    y_tr_sc = sc_y.fit_transform(y_train.to_frame())
    y_ts_sc = sc_y.transform(y_test.to_frame())
    
    input_columns = torch.tensor(X_tr_sc).type(torch.float32)
    output_columns = torch.tensor(y_tr_sc).type(torch.float32)

    data = TensorDataset(input_columns,output_columns)
    split = 0.3
    num_rows = list(input_columns.shape)[0]
    val_split = int(num_rows * split)
    train_split = num_rows - val_split
    train_set,val_set = random_split(data, [train_split,val_split])

    train_loader = DataLoader(train_set, 100, shuffle = True,drop_last = True)
    val_loader = DataLoader(val_set)
    return train_loader,val_loader,input_columns,output_columns,X,y,X_ts_sc,y_ts_sc,sc_y

In [17]:
def define_nn_optuna(trial,input_columns):
    filename = var.get()
    dense = float(ro.get())
    new_df = asad_plotting(filename,dense)[0]
    input_columns = inputs_nn(new_df)[2]
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers",1,3)
    layers = []
    in_features = input_columns.shape[1] #inputs_nn(filename = var.get(),density=float(ro.get()),columns_values = ['SWOB','TFLO','SPPA'])[2]
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i),10,56)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i),0.2,0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [18]:
def objective_nn(trial):
    filename = var.get()
    dense = float(ro.get())
    new_df = asad_plotting(filename,dense)[0]
    train_loader = inputs_nn(new_df)[0]
    val_loader = inputs_nn(new_df)[1]
    input_columns = inputs_nn(new_df)[2]
    output_columns = inputs_nn(new_df)[3]
    X = inputs_nn(new_df)[4]
    y = inputs_nn(new_df)[5]
    EPOCHS = 21
    # Generate the model.
    model = define_nn_optuna(trial,input_columns)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    #X = inputs_nn(filename = var.get(),density=float(ro.get()),columns_values = ['SWOB','TFLO','SPPA'])[4]
    #y = inputs_nn(filename = var.get(),density=float(ro.get()),columns_values = ['SWOB','TFLO','SPPA'])[5]
    
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (X, y) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(X)
            loss = F.mse_loss(output, y)
            loss.backward()
            optimizer.step()
            
        model.eval()
        loss_val = []
        criterion = []
        with torch.no_grad():
            for batch_idx, (X, y) in enumerate(val_loader):
                output = model(X)
                loss = F.mse_loss(output, y)
                loss_val.append(loss)
        error = np.mean(loss_val)
        trial.report(error, epoch)
        
    return error

In [19]:
def best_nn(*args):
    #filename = var.get()
    #dense = float(ro.get())
    #new_df = asad_plotting(filename,['SWOB','TFLO','SPPA'],dense)[0]
    study_nn = optuna.create_study(direction="minimize")
    study_nn.optimize(objective_nn, n_trials=6, timeout=600)
    best_param = study_nn.best_params
    #num_layers.set(best_param['n_layers'])
    #optimizer_name.set(best_param['optimizer'])
    #learning_rate.set(best_param['lr'])
    #n_units
    finish_nn.set(best_param)
    ready_nn.set('Neural Network is Optimized')
    #return best_param

In [20]:
def init_NN(input_columns):
    best_param = finish_nn.get()
    best_param = ast.literal_eval(best_param)
    n_layers = best_param['n_layers']
    layers = []

    in_features = input_columns.shape[1]
    for i in range(n_layers):
        out_features = best_param['n_units_l{}'.format(i)]
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = best_param['dropout_l{}'.format(i)]
        layers.append(nn.Dropout(p))
        in_features = out_features
    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [21]:
def train_nn():
    filename = var.get()
    dense = float(ro.get())
    new_df = asad_plotting(filename,dense)[0]
    train_loader = inputs_nn(new_df)[0]
    val_loader = inputs_nn(new_df)[1]
    input_columns = inputs_nn(new_df)[2]
    output_columns = inputs_nn(new_df)[3]
    X = inputs_nn(new_df)[4]
    y = inputs_nn(new_df)[5]
    X_ts_sc = inputs_nn(new_df)[6]
    y_ts_sc = inputs_nn(new_df)[7]
    sc_y  = inputs_nn(new_df)[8]
    best_param = finish_nn.get()
    best_param = ast.literal_eval(best_param)
    new_nn = init_NN(input_columns)
    optimizer_name = best_param['optimizer']
    lr = best_param['lr']
    optimizer = getattr(optim, optimizer_name)(new_nn.parameters(), lr=lr)
    
    train_loss = []
    val_loss = []
    
    #X = inputs_nn(filename = var.get(),density=float(ro.get()),columns_values = ['SWOB','TFLO','SPPA'])[4]
    #y = inputs_nn(filename = var.get(),density=float(ro.get()),columns_values = ['SWOB','TFLO','SPPA'])[5]
    EPOCHS = 21
    for epoch in range(EPOCHS):
        loss_train = []
        new_nn.train()
        for batch_idx, (X, y) in enumerate(train_loader):
            optimizer.zero_grad()
            output = new_nn(X)
            loss = F.mse_loss(output, y)
            loss_sqrt = torch.sqrt(loss)
            loss_train.append(loss_sqrt.detach().item())
            loss.backward()
            optimizer.step()
        train_loss.append(np.mean(loss_train))

        new_nn.eval()
        loss_val = []
        criterion = []
        with torch.no_grad():
            for batch_idx, (X, y) in enumerate(val_loader):
                output = new_nn(X)
                loss = F.mse_loss(output, y)
                loss_val.append(torch.sqrt(loss))
        error = np.mean(loss_val)
        val_loss.append(np.mean(loss_val))
        #print(f'Validation RMSE in epoch {epoch} is {error}')

    test_set_sc = new_nn(torch.tensor(X_ts_sc).type(torch.float32))
    test_set = sc_y.inverse_transform(test_set_sc.detach().numpy())
    y_test = sc_y.inverse_transform(y_ts_sc)
    #print('For ANN the error on test set is {}'.format(sqrt(mse(test_set,y_test))))
    criterion_nn = r2_score(test_set,y_test)
    error = sqrt(mse(test_set,y_test))
    return criterion_nn,error,train_loss,val_loss,test_set,y_test

In [22]:
def nn_performance():
    criterion_nn = train_nn()[0]
    error = train_nn()[1]
    train_loss = train_nn()[2]
    val_loss = train_nn()[3]
    test_set = train_nn()[4]
    y_test = train_nn()[5]
    
    rmse_nn.set("{:.4f}".format(sqrt(mse(test_set,y_test))))
    R_square_nn.set("{:.4f}".format(criterion_nn))
    
    plot1 = plt.subplot(2, 2, 1) # divide as 2x2, plot top left
    plot2 = plt.subplot(2, 2, 2) # divide as 2x2, plot top right
    plot3 = plt.subplot(2, 1, 2) # divide as 2x1, plot bottom
    
    plot1.plot(train_loss,c ='r',label = 'Train Loss')
    plot1.plot(val_loss,c='g',label = 'Validation Loss')
    plot1.set_xlabel("EPOCH")
    plot1.set_ylabel("RMSE")
    plot1.set_title("RMSE Loss function")
    plot1.legend()
    
    plot2.scatter(test_set,y_test,c = 'r')
    plot2.set_xlabel('Actual Values')
    plot2.set_ylabel('Predicted Values')
    plot2.plot(np.unique(y_test.flatten()), np.poly1d(np.polyfit(y_test.flatten(), test_set.flatten(), 1))(np.unique(y_test.flatten())))
    plot2.set_title('ANN R^2 {}'.format(criterion_nn))
    plot3.legend()
    
    plot3.plot(test_set.flatten(),c ='r',label = 'Predictions')
    plot3.plot(y_test.flatten(),c='g',label = 'Real Value')
    plot3.set_xlabel("Number")
    plot3.set_ylabel("SPPA")
    plot3.legend()
    plot3.set_title("Predictions vs Real Values by ANN")
    plt.tight_layout()
    plt.show()

In [23]:
def drill_string_loss_new(Q,MD,density):
    try:
        tau = float(t.get())
        theta600 = float(th600.get())
        theta300 = float(th300.get())
        #D = float(Diameter.get())
        Yp = float(Yield.get())
        dc_length = float(length_dc.get())
        lwd_length = float(length_lwd.get())
        mwd_length = float(length_mwd.get())
        motor_length = float(length_motor.get())
        dc_D = float(D_dc.get())
        dp_D = float(D_dp.get())
        
        #Drill pipe Length
        dp_length = MD - dc_length - mwd_length - motor_length - lwd_length
        #drill_pipe_length.set("{:.2f}".format(dp_length))
        #Mud motor
        Pfm = 87
        #For the MWD
        Pfmwd = density*Q/29000
        #LWD
        Pflwd = density*(Q**2)/121000
        
        #For drill collar
        
        n = 3.322*log10((theta600 - tau)/(theta300 - tau))
        K = 510*(theta300 - tau)/(511**n)
        velocity = Q/(2.448*(dc_D**2))
        C = 1 - (1/(2*n+1))*(Yp/(Yp + 0.00208*K*((9.75*Q*(3*n+1))/(n*(dc_D**3)))**n))
        Nre = (2*(3*n+1)/n)*((7.48*density*velocity**(2-n)*((dc_D/24)**n))/(Yp*((dc_D/(24*velocity))**n) + 0.00208*K*(((3*n+1)/(n*C))**n)))
        gamma = (math.log(n) + 3.93)/50
        z = (1.75-math.log(n))/7
        Nrec = ((8*(2*n+1))/(n*gamma))**(1/(1-z))
        if Nre <= Nrec:
            friction_loss_dc = ((0.09984*K)/(14400*dc_D))*(Yp/(0.00208*K)+((3*n+1)/(n*C)*(9.76*Q)/(dc_D**3))**n)*dc_length
        else:
            f = gamma*((C*Nre)**(-z))
            friction_loss_dc = (7.48*f*((0.002217*Q)**2)*density)/(0.005712*(dc_D**5))*dc_length
        
        #For drill pipe
        velocity = Q/(2.448*(dp_D**2))
        C = 1 - (1/(2*n+1))*(Yp/(Yp + 0.00208*K*((9.75*Q*(3*n+1))/(n*(dp_D**3)))**n))
        Nre = (2*(3*n+1)/n)*((7.48*density*velocity**(2-n)*((dp_D/24)**n))/(Yp*((dp_D/(24*velocity))**n) + 0.00208*K*(((3*n+1)/(n*C))**n)))
        gamma = (math.log(n) + 3.93)/50
        z = (1.75-math.log(n))/7
        Nrec = ((8*(2*n+1))/(n*gamma))**(1/(1-z))
        if Nre <= Nrec:
            friction_loss_dp = ((0.09984*K)/(14400*dp_D))*(Yp/(0.00208*K)+((3*n+1)/(n*C)*(9.76*Q)/(dp_D**3))**n)*dp_length
        else:
            f = gamma*((C*Nre)**(-z))
            friction_loss_dp = (7.48*f*((0.002217*Q)**2)*density)/(0.005712*(dp_D**5))*dp_length
            
        friction_loss = friction_loss_dp + friction_loss_dc + Pfm + Pfmwd + Pflwd
    except ValueError:
        pass
    
        #For drill string
    return friction_loss

In [24]:
def drill_string_length():
    check = int(unit.get())
    filename = var.get()
    density = float(ro.get())
    depth = asad_plotting(filename,density)[1]
    #print(depth)
    dc_length = float(length_dc.get())
    lwd_length = float(length_lwd.get())
    mwd_length = float(length_mwd.get())
    motor_length = float(length_motor.get())
    if check == 0:
        dp_length = depth - dc_length - mwd_length - motor_length - lwd_length
        drill_pipe_length.set("{:.2f}".format(dp_length))
    elif check == 1:
        dp_length = 0.3048*depth - dc_length - mwd_length - motor_length - lwd_length
        drill_pipe_length.set("{:.2f}".format(dp_length))

    

In [25]:
def hole_cleaning_effect():
    ROP_val = float(ROP.get())
    D_hole_val = float(D_hole.get())
    D_dc_ex_val = float(D_dc_ex.get())
    ESV_val = float(ESV.get())
    Yp_val = float(Yield.get())
    visc_p_val = float(visc_p.get())
    angle_val = float(angle.get())
    D50_val = float(D50.get())
    density_val = float(ro.get())
    epsilon = 0.01
    C_c = C_conc(ROP_val)
    V_c = V_cut(C_c,D_hole_val,D_dc_ex_val,ROP_val)
    while True:
        V_crit = V_critical(V_c,ESV_val)
        visc_ap = visc_app(Yp_val,D_hole_val,D_dc_ex_val,visc_p_val,V_crit)
        ESV_tr = ESV_true(visc_ap)
        if ESV_tr - ESV_val <= epsilon:
            break
        else:
            ESV_val = ESV_tr
    C_an = C_ang(angle_val)
    C_s = C_size(D50_val)
    C_dens = C_mw(density_val)
    ESV_g = ESV_gen(C_an,C_s,C_dens,ESV_tr)
    V_crit = V_critical(V_c,ESV_g)
    Q = flow_rate(D_hole_val,D_dc_ex_val,V_crit)
    Q_hole.set("{:.2f}".format(Q))             

In [26]:
def objective_cat(trial):
    filename = var.get()
    dense = 10.2
    new_df = asad_plotting(filename,dense)[0]
    X = new_df[['SWOB','TFLO']]
    y = new_df['SPPA']
    
    param = {
        'iterations':500,
        'use_best_model':True,
        'od_type' : "Iter",
        'od_wait' : 100,
#         'random_seed': 240,
#          "scale_pos_weight":trial.suggest_int("scale_pos_weight", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
        "depth": trial.suggest_int("max_depth", 1, 3),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
          'eval_metric':trial.suggest_categorical("eval_metric",['MAE','RMSE']),
         #'one_hot_max_size':1024
        }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    model=CatBoostRegressor(**param)
    model.fit(X_train,y_train,eval_set = (X_test,y_test), silent = True)
    preds = model.predict(X_test)
    criterion = r2_score(preds,y_test)
    return criterion

In [27]:
def best_cat():
    study_cat = optuna.create_study(direction="maximize")
    study_cat.optimize(objective_cat, n_trials=6, timeout=600)
    best_params = study_cat.best_params
    finish_cat.set(best_params)
    ready_cat.set("Boosting is Optimized")
    return best_params

In [28]:
def train_cat():
    filename = var.get()
    dense = 10.2
    new_df = asad_plotting(filename,dense)[0]
    X = new_df[['SWOB','TFLO']]
    y = new_df['SPPA']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    folds = 4
    shuffle = True
    kf = KFold(n_splits=folds)
    i = 0
    best_params = finish_cat.get()
    best_params = ast.literal_eval(best_params)
    booster=CatBoostRegressor(**best_params)
    val_loss = []
    for train_index, val_index in kf.split(X_train):
        i+=1
        X_t, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_t, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        booster.fit(X_t,y_t,eval_set = (X_val,y_val), silent = True)
        preds = booster.predict(X_val)
        error=sqrt(mse(preds,y_val))
        val_loss.append(error)
        #print('RMSE in Fold {} is {}'.format(i,error))
        criterion = r2_score(preds,y_val)
        #print('R^2 in Fold {} is {}'.format(i,criterion))

    test_preds = booster.predict(X_test)
    error_test = sqrt(mse(test_preds,y_test))
    criterion_boost = r2_score(test_preds,y_test)
    #rmse_cat.set(error_test)
    #R_square_cat.set(criterion_boost)
    #print('RMSE on test_set is {}'.format(error_test))
    #print('R^2 on test_set is {}'.format(criterion_boost))
    return booster,criterion_boost,error_test,val_loss,test_preds,y_test


In [29]:
def train_cat_annuli():
    filename = var.get()
    dense = float(ro.get())
    new_df = asad_plotting(filename,dense)[0]
    X = new_df[['DEPTH','TFLO']]
    y = new_df['DHAP']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    folds = 4
    shuffle = True
    kf = KFold(n_splits=folds)
    i = 0
    booster=CatBoostRegressor()
    val_loss = []
    for train_index, val_index in kf.split(X_train):
        i+=1
        X_t, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_t, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        booster.fit(X_t,y_t,eval_set = (X_val,y_val), silent = True)
        preds = booster.predict(X_val)
        error=sqrt(mse(preds,y_val))
        val_loss.append(error)
        #print('RMSE in Fold {} is {}'.format(i,error))
        criterion = r2_score(preds,y_val)
        #print('R^2 in Fold {} is {}'.format(i,criterion))

    test_preds = booster.predict(X_test)
    error_test = sqrt(mse(test_preds,y_test))
    criterion_boost = r2_score(test_preds,y_test)
    #rmse_cat.set(error_test)
    #R_square_cat.set(criterion_boost)
    #print('RMSE on test_set is {}'.format(error_test))
    #print('R^2 on test_set is {}'.format(criterion_boost))
    return booster,criterion_boost,error_test,val_loss,test_preds,y_test


In [30]:
def cat_performance():
    criterion_boost = train_cat()[1]
    error_test = train_cat()[2]
    val_loss = train_cat()[3]
    test_preds = train_cat()[4] 
    y_test = train_cat()[5]
    rmse_cat.set("{:.4f}".format(error_test))
    R_square_cat.set("{:.4f}".format(criterion_boost))
    
    plt.plot()
    plot1 = plt.subplot(2, 2, 1) # divide as 2x2, plot top left
    plot2 = plt.subplot(2, 2, 2) # divide as 2x2, plot top right
    plot3 = plt.subplot(2, 1, 2) # divide as 2x1, plot bottom

    plot1.plot(val_loss,c='g',label = 'Validation Loss')
    plot1.set_xlabel("Fold")
    plot1.set_ylabel("RMSE")
    plot1.set_title("RMSE Loss function")

    plot2.scatter(test_preds,y_test,color = 'r')
    plot2.set_xlabel('Actual values')
    plot2.set_ylabel('Predicted values')
    plot2.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, test_preds, 1))(np.unique(y_test)))
    plot2.set_title('Boosting R^2 {}'.format(criterion_boost))
    plot2.legend()

    plot3.plot(test_preds,c ='r',label = 'Predictions')
    plot3.plot(y_test.reset_index(drop = True),c='g',label = 'Real Value')
    plt.xlabel("Number")
    plt.ylabel("SPPA")
    plt.legend()
    plt.title("Predictions vs Real Values by Boosting")
    plt.tight_layout()
    plt.show()

In [31]:
def cat_performance_annuli():
    criterion_boost_ann = train_cat_annuli()[1]
    error_test_ann = train_cat_annuli()[2]
    val_loss_ann = train_cat_annuli()[3]
    test_preds = train_cat_annuli()[4] 
    y_test = train_cat_annuli()[5]
    #rmse_cat.set("{:.4f}".format(error_test))
    #R_square_cat.set("{:.4f}".format(criterion_boost))
    
    plt.plot()
    plot1 = plt.subplot(2, 2, 1) # divide as 2x2, plot top left
    plot2 = plt.subplot(2, 2, 2) # divide as 2x2, plot top right
    plot3 = plt.subplot(2, 1, 2) # divide as 2x1, plot bottom

    plot1.plot(val_loss_ann,c='g',label = 'Validation Loss')
    plot1.set_xlabel("Fold")
    plot1.set_ylabel("RMSE")
    plot1.set_title("RMSE Loss function")

    plot2.scatter(test_preds,y_test,color = 'r')
    plot2.set_xlabel('Actual values')
    plot2.set_ylabel('Predicted values')
    plot2.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, test_preds, 1))(np.unique(y_test)))
    plot2.set_title('Boosting R^2 {}'.format(criterion_boost_ann))
    plot2.legend()

    plot3.plot(test_preds,c ='r',label = 'Predictions')
    plot3.plot(y_test.reset_index(drop = True),c='g',label = 'Real Value')
    plt.xlabel("Number")
    plt.ylabel("DHAP")
    plt.legend()
    plt.title("Predictions vs Real Values by Boosting")
    plt.tight_layout()
    plt.show()

In [32]:
def calculate_ds_loss_new(equation_inputs,density,MD):
    ds_loss = []
    for rate in equation_inputs[:,-1]:
        loss = drill_string_loss_new(rate,MD,density)
        ds_loss.append(loss)
    ds_loss = np.asarray(ds_loss).reshape(-1,1)
    return ds_loss


def calculate_annulus_loss(model,annulus_inputs,depth,density):
    annulus_pressure_hydro = model.predict(annulus_inputs)
    annulus_losses = abs(annulus_pressure_hydro - 0.052*density*depth)
    return annulus_losses



def cal_pop_fitness(equation_inputs,ds_loss,annulus_loss,model):
    sppa = model.predict(equation_inputs)
    sppa = sppa.reshape(-1,1)
    annulus_loss = annulus_loss.reshape(-1,1)
    #print('SPPA {}'.format(sppa))
    bit_loss = sppa - ds_loss - annulus_loss
    #print('Bit_loss {}'.format(bit_loss))
    ratio = np.divide(bit_loss,sppa)
    fitness = 0.6 - bit_loss/sppa.reshape(-1,1)
    return fitness,ratio

def select_mating_pool(equation_inputs, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, equation_inputs.shape[1]))
    for parent_num in range(num_parents):
        min_fitness_idx = np.where(fitness == np.min(fitness))
        min_fitness_idx = min_fitness_idx[0][0]
        parents[parent_num, :] = equation_inputs[min_fitness_idx, :]
        fitness[min_fitness_idx] = +99999999999
    return parents

def crossover(parents, offspring_size):
#print('Parents are {}'.format(parents))
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover,lim_wob_min,lim_wob_max,lim_flow_min,lim_flow_max):
    #print('Offspring_crossover before mutation is {}'.format(offspring_crossover))
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        if offspring_crossover[idx, 0]<=lim_wob_min or offspring_crossover[idx, 1]<=lim_flow_min:
            offspring_crossover[idx, 0] = abs(offspring_crossover[idx, 0] + np.random.uniform(0, 3.0, 1))
            offspring_crossover[idx, 1] = abs(offspring_crossover[idx, 1] + np.random.uniform(0.0, 20.0, 1))
        elif offspring_crossover[idx, 0]>=lim_wob_max or offspring_crossover[idx, 1]>=lim_flow_max:
            offspring_crossover[idx, 0] = abs(offspring_crossover[idx, 0] + np.random.uniform(-2.0, -1.0, 1))
            offspring_crossover[idx, 1] = abs(offspring_crossover[idx, 1] + np.random.uniform(-20.0, -10.0, 1))
        else:
            offspring_crossover[idx, 0] = abs(offspring_crossover[idx, 0] + np.random.uniform(-1.0, 3.0, 1))
            offspring_crossover[idx, 1] = abs(offspring_crossover[idx, 1] + np.random.uniform(-20.0, 20.0, 1))
        #print('Offspring_crossover after mutation is {}'.format(offspring_crossover))
    return offspring_crossover

def genetic_algorithm():
    
    my_list = []
    my_list_ratio = []
    filename = var.get()
    density = float(ro.get())
    new_df = asad_plotting(filename,density)[0]
    best_params = best_cat()
    #best_params = ast.literal_eval(best_params)
    model1=train_cat()[0]
    model2=train_cat_annuli()[0]
    number_combinations = int(num_combinations.get())
    lower_limit = float(criterion_lower.get())
    upper_limit = float(criterion_upper.get())
    
    for combination in range(1,number_combinations+1):
        TFLO_min = float(min_flow.get())
        TFLO_max = float(max_flow.get())
        WOB_min =float(min_wob.get())
        WOB_max = float(max_wob.get())

        flow_rate = random.uniform(TFLO_min,TFLO_max)
        wob = random.uniform(WOB_min,WOB_max)
        depth = asad_plotting(filename,density)[1]
        depth_array = np.empty(6)
        depth_array.fill(depth)
        #annulus_loss = asad_plotting(filename,['SWOB','TFLO','SPPA'],density)[2]
        num_parents_mating = 3
        epoch = 0
        while True:
            gen1 = np.array([random.uniform(WOB_min,WOB_max),random.uniform(TFLO_min,TFLO_max)])
            gen2 = np.array([random.uniform(WOB_min,WOB_max),random.uniform(TFLO_min,TFLO_max)])
            gen3 = np.array([random.uniform(WOB_min,WOB_max),random.uniform(TFLO_min,TFLO_max)])
            gen4 = np.array([random.uniform(WOB_min,WOB_max),random.uniform(TFLO_min,TFLO_max)])
            gen5 = np.array([random.uniform(WOB_min,WOB_max),random.uniform(TFLO_min,TFLO_max)])
            gen6 = np.array([random.uniform(WOB_min,WOB_max),random.uniform(TFLO_min,TFLO_max)])
            new_population = np.vstack((gen1,gen2,gen3,gen4,gen5,gen6))
            #print('New population {}'.format(new_population))
            equation_inputs_size = new_population.shape
            epoch += 1
            #print(f'Epoch number is {epoch}')
            i = 0
            while True:
                annulus_loss_array = np.vstack((depth_array,new_population[:,1] )).T
                annulus_loss = calculate_annulus_loss(model2,annulus_loss_array,depth,density)
                i+=1
                #print('Iteration {}'.format(i))
                target_value = calculate_ds_loss_new(new_population,density,depth)
                
                #print('Drill string loss {}'.format(target_value))
                fitness = cal_pop_fitness(new_population,target_value,annulus_loss,model1)[0]
                ratio = cal_pop_fitness(new_population,target_value,annulus_loss,model1)[1]
                #print('Fitness_value {}'.format(fitness))
                #print('Ratio {}'.format(ratio))
                if np.any(( ratio <=upper_limit) & (ratio >=lower_limit)) or i >=1000:
                    break

                # Selecting the best parents in the population for mating.
                parents = select_mating_pool(new_population, fitness, 
                                                  num_parents_mating)
                #print('parents {}'.format(parents))

                # Generating next generation using crossover.
                offspring_crossover = crossover(parents,
                                                   offspring_size=(equation_inputs_size[0]-parents.shape[0], equation_inputs_size[1]))
                #print('offspring_crossover {}'.format(offspring_crossover))

                # Adding some variations to the offsrping using mutation.
                offspring_mutation = mutation(offspring_crossover,WOB_min,WOB_max,TFLO_min,TFLO_max)
                #print('Mutation {}'.format(offspring_mutation))

                # Creating the new population based on the parents and offspring.
                new_population[0:parents.shape[0], :] = parents
                new_population[parents.shape[0]:, :] = offspring_mutation
            best_ratio = ratio[(ratio >= lower_limit) & (ratio <= upper_limit)]
            if best_ratio.size == 0 and epoch < 30:
                continue
            else:
                break
        if epoch>=30:
            print(f'The last ratio  is {ratio}')
            best_match_idx = np.where(ratio == np.min(ratio))[0][0]
            best_solution = new_population[best_match_idx, :]
            print('WOB {}'.format(best_solution[-2]))
            print('Flow rate {}'.format(best_solution[-1]))
        else:
            #print('For {} combination desired ratio values {}'.format(combination,best_ratio[0]))
            my_list_ratio.append(best_ratio[0])
            best_match_idx = np.where(ratio == np.min(best_ratio))[0][0]
            best_solution = new_population[best_match_idx, :]
            my_list.append(best_solution)
            print('For {} combination best WOB {}'.format(combination,best_solution[-2]))
            print('For {} combination best flow_rate {}\n'.format(combination,best_solution[-1]))
    if number_combinations == 1:
        wob1.set("{:.2f}".format(my_list[0][0]))
        flow1.set("{:.2f}".format(my_list[0][1]))
        best_ratio1.set("{:.2f}".format(my_list_ratio[0]))
    elif number_combinations == 2:
        wob1.set("{:.2f}".format(my_list[0][0]))
        wob2.set("{:.2f}".format(my_list[1][0]))
        flow1.set("{:.2f}".format(my_list[0][1]))
        flow2.set("{:.2f}".format(my_list[1][1]))
        best_ratio1.set("{:.2f}".format(my_list_ratio[0]))
        best_ratio2.set("{:.2f}".format(my_list_ratio[1]))
    elif number_combinations == 3:
        wob1.set("{:.2f}".format(my_list[0][0]))
        wob2.set("{:.2f}".format(my_list[1][0]))
        wob3.set("{:.2f}".format(my_list[2][0]))
        flow1.set("{:.2f}".format(my_list[0][1]))
        flow2.set("{:.2f}".format(my_list[1][1]))
        flow3.set("{:.2f}".format(my_list[2][1]))
        best_ratio1.set("{:.2f}".format(my_list_ratio[0]))
        best_ratio2.set("{:.2f}".format(my_list_ratio[1]))
        best_ratio3.set("{:.2f}".format(my_list_ratio[2]))
    elif number_combinations == 4:
        wob1.set("{:.2f}".format(my_list[0][0]))
        wob2.set("{:.2f}".format(my_list[1][0]))
        wob3.set("{:.2f}".format(my_list[2][0]))
        wob4.set("{:.2f}".format(my_list[3][0]))
        flow1.set("{:.2f}".format(my_list[0][1]))
        flow2.set("{:.2f}".format(my_list[1][1]))
        flow3.set("{:.2f}".format(my_list[2][1]))
        flow4.set("{:.2f}".format(my_list[3][1]))
        best_ratio1.set("{:.2f}".format(my_list_ratio[0]))
        best_ratio2.set("{:.2f}".format(my_list_ratio[1]))
        best_ratio3.set("{:.2f}".format(my_list_ratio[2]))
        best_ratio4.set("{:.2f}".format(my_list_ratio[3]))
    else:
        wob1.set("{:.2f}".format(my_list[0][0]))
        wob2.set("{:.2f}".format(my_list[1][0]))
        wob3.set("{:.2f}".format(my_list[2][0]))
        wob4.set("{:.2f}".format(my_list[3][0]))
        wob5.set("{:.2f}".format(my_list[4][0]))
        flow1.set("{:.2f}".format(my_list[0][1]))
        flow2.set("{:.2f}".format(my_list[1][1]))
        flow3.set("{:.2f}".format(my_list[2][1]))
        flow4.set("{:.2f}".format(my_list[3][1]))
        flow5.set("{:.2f}".format(my_list[4][1]))
        best_ratio1.set("{:.3f}".format(my_list_ratio[0]))
        best_ratio2.set("{:.3f}".format(my_list_ratio[1]))
        best_ratio3.set("{:.3f}".format(my_list_ratio[2]))
        best_ratio4.set("{:.3f}".format(my_list_ratio[3]))
        best_ratio5.set("{:.3f}".format(my_list_ratio[4]))

In [33]:
def define_limits():
    filename = var.get()
    density = float(ro.get())
    new_df = asad_plotting(filename,density)[0]
    
    TFLO_min = new_df[new_df['TFLO'] > 600]['TFLO'].min()
    TFLO_max = new_df['TFLO'].max()
    WOB_min = new_df[new_df['SWOB']>20]['SWOB'].min()
    WOB_max = new_df['SWOB'].max()
    min_wob.set("{:.2f}".format(WOB_min))
    max_wob.set("{:.2f}".format(WOB_max))
    min_flow.set("{:.2f}".format(TFLO_min))
    max_flow.set("{:.2f}".format(TFLO_max))

In [34]:
def close_window (): 
    root.destroy()

In [35]:
%matplotlib qt
# import tkinter module
from tkinter import * 
from tkinter.ttk import *
  
# creating main tkinter window/toplevel
root = Tk()
root.geometry("1200x620")
mainframe = ttk.Frame(root)

mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

#Declare Variables

var=StringVar()
t = StringVar()
th600 = StringVar()
th300 = StringVar()
rate = StringVar()
Diameter = StringVar()
Yield = StringVar()
ro = StringVar()
angle = StringVar()
visc_p = StringVar()
D50 = StringVar()
length_dc = StringVar()
length_lwd = StringVar()
length_mwd = StringVar()
length_motor = StringVar()
D_dc = StringVar()
D_dp = StringVar()
criterion_lower = StringVar()
criterion_upper = StringVar()
num_combinations = StringVar()
drill_pipe_length = StringVar()
D_dc_ex = StringVar()
D_hole = StringVar()
ROP = StringVar()
ESV = StringVar()
Q_hole = StringVar()

depth = StringVar()
Pf = StringVar()
finish_nn = StringVar()
rmse_nn = StringVar()
R_square_nn = StringVar()
finish_cat = StringVar()
rmse_cat = StringVar()
R_square_cat = StringVar()
ready_nn = StringVar()
ready_cat = StringVar()

#limits
min_wob = StringVar()
max_wob = StringVar()
min_flow = StringVar()
max_flow = StringVar()

#values
wob1 = StringVar()
flow1 = StringVar()
wob2 = StringVar()
flow2 = StringVar()
wob3 = StringVar()
flow3 = StringVar()
wob4 = StringVar()
flow4 = StringVar()
wob5 = StringVar()
flow5 =  StringVar()

best_ratio1 = StringVar()
best_ratio2 = StringVar()
best_ratio3 = StringVar()
best_ratio4 = StringVar()
best_ratio5 = StringVar()

unit = StringVar()


#Insert inputs
ttk.Label(mainframe, text="Tau",font=("Arial", 10)).grid(column=1, row=2,pady = 4, padx = 4)
ttk.Label(mainframe, text="Theta600",font=("Arial", 10)).grid(column=1, row=3,pady = 4,padx = 4)
ttk.Label(mainframe, text="Theta300",font=("Arial", 10)).grid(column=1, row=4,pady = 4,padx = 4)
#ttk.Label(mainframe, text="Diameter",font=("Arial", 10)).grid(column=1, row=5,pady = 2,padx = 2, sticky=W)
ttk.Label(mainframe, text="Yield Point",font=("Arial", 10)).grid(column=1, row=5,pady = 4,padx = 4)
ttk.Label(mainframe, text="Density",font=("Arial", 10)).grid(column=1, row=6,pady = 4,padx = 4)
ttk.Label(mainframe, text="Plastic Viscosity",font=("Arial", 10)).grid(column=1, row=7,pady = 4,padx = 4)
ttk.Label(mainframe, text="Well Angle",font=("Arial", 10)).grid(column=1, row=8,pady = 4,padx = 4)
ttk.Label(mainframe, text="Average Cutting Size",font=("Arial", 10)).grid(column=1, row=9,pady = 4,padx = 4)


ttk.Label(mainframe, text="DC Length",font=("Arial", 10)).grid(column=4, row=2,pady = 4, padx = 6)
ttk.Label(mainframe, text="LWD Length",font=("Arial", 10)).grid(column=4, row=3,pady = 4,padx = 6)
ttk.Label(mainframe, text="MWD Length",font=("Arial", 10)).grid(column=4, row=4,pady = 4,padx = 6)
ttk.Label(mainframe, text="Mud Motor Length",font=("Arial", 10)).grid(column=4, row=5,pady = 4,padx = 6)
ttk.Label(mainframe, text="DC Inside Diameter",font=("Arial", 10)).grid(column=4, row=6,pady = 4,padx = 6)
ttk.Label(mainframe, text="DP Inside Diameter",font=("Arial", 10)).grid(column=4, row=7,pady = 4,padx = 6)
ttk.Label(mainframe, text="DC Outside Diameter",font=("Arial", 10)).grid(column=4, row=8,pady = 4,padx = 6)
ttk.Label(mainframe, text="Hole Diameter",font=("Arial", 10)).grid(column=4, row=9,pady = 4,padx = 6)
#ttk.Label(mainframe, text="DP Length",font=("Arial", 10)).grid(column=4, row=8,pady = 4,padx = 6)
ttk.Button(mainframe, text="Calculate DP Length", command=drill_string_length).grid(column=4, row=12, pady = 4, padx = 4)
ttk.Label(mainframe,background="white", textvariable=drill_pipe_length).grid(column=5, row=12,pady = 4, padx = 4)



def field_unit():
    a = 0
    unit.set(a)
    global label1
    global label2
    global label3
    global label4
    global label5
    global label6
    global label7
    global label8
    global label9
    global label10
    global label11
    global label12
    global label13
    global label14
    global label15
    global label16
    global label17
    global label18
    global label19
    global label20
    global label21
    global label22
    global label23
    global label24
    global label25
    global label26
    global label53
    global label54
    global label55
    global label56
    global label57
    global label63
    
    
    label1 = ttk.Label(mainframe, text="lbf/100_ft2",font=("Arial", 10))
    label1.grid(column=3, row=2,pady = 4, padx = 4)
    label2 = ttk.Label(mainframe, text="lbf/100_ft2",font=("Arial", 10))
    label2.grid(column=3, row=3,pady = 4,padx = 4)
    label3 = ttk.Label(mainframe, text="lbf/100_ft2",font=("Arial", 10))
    label3.grid(column=3, row=4,pady = 4,padx = 4)
    #ttk.Label(mainframe, text="inch",font=("Arial", 10)).grid(column=3, row=5,pady = 2,padx = 2, sticky=W)
    label4 = ttk.Label(mainframe, text="lbf/100_ft2",font=("Arial", 10))
    label4.grid(column=3, row=5,pady = 4,padx = 4)
    label5 = ttk.Label(mainframe, text="ppg",font=("Arial", 10))
    label5.grid(column=3, row=6,pady = 4,padx = 4)
    label53 = ttk.Label(mainframe, text="cP",font=("Arial", 10))
    label53.grid(column=3, row=7,pady = 4,padx = 4)
    label54 = ttk.Label(mainframe, text="degree",font=("Arial", 10))
    label54.grid(column=3, row=8,pady = 4,padx = 4)
    label55 = ttk.Label(mainframe, text="inch",font=("Arial", 10))
    label55.grid(column=3, row=9,pady = 4,padx = 4)
    
    label6 = ttk.Label(mainframe, text="ft",font=("Arial", 10))
    label6.grid(column=6, row=2,pady = 4, padx = 4)
    label7 = ttk.Label(mainframe, text="ft",font=("Arial", 10))
    label7.grid(column=6, row=3,pady = 4,padx = 4)
    label8 = ttk.Label(mainframe, text="ft",font=("Arial", 10))
    label8.grid(column=6, row=4,pady = 4,padx = 4)
    label9 = ttk.Label(mainframe, text="ft",font=("Arial", 10))
    label9.grid(column=6, row=5,pady = 4,padx = 4)
    label10 = ttk.Label(mainframe, text="inch",font=("Arial", 10))
    label10.grid(column=6, row=6,pady = 4,padx = 4)
    label11 = ttk.Label(mainframe, text="inch",font=("Arial", 10))
    label11.grid(column=6, row=7,pady = 4,padx = 4)
    label12 = ttk.Label(mainframe, text="inch",font=("Arial", 10))
    label12.grid(column=6, row=8,pady = 4,padx = 4)
    label56 = ttk.Label(mainframe, text="inch",font=("Arial", 10))
    label56.grid(column=6, row=9,pady = 4,padx = 4)
    label57 = ttk.Label(mainframe, text="ft",font=("Arial", 10))
    label57.grid(column=6, row=12,pady = 4,padx = 4)
    label63 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label63.grid(column=12, row=29,pady = 4, padx = 4)
    #label58 = ttk.Label(mainframe, text="ft",font=("Arial", 10))
    #label58.grid(column=6, row=12,pady = 4,padx = 4)

    label13 = ttk.Label(mainframe, text="1000lbf",font=("Arial", 10))
    label13.grid(column=3, row=21,pady = 4, padx = 4)
    label14 = ttk.Label(mainframe, text="1000lbf",font=("Arial", 10))
    label14.grid(column=3, row=22,pady = 4,padx = 4)
    label15 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label15.grid(column=3, row=23,pady = 4,padx = 4)
    label16 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label16.grid(column=3, row=24,pady = 4,padx = 4)

    label17 = ttk.Label(mainframe, text="1000lbf",font=("Arial", 10))
    label17.grid(column=4, row=28,pady = 4, padx = 4)
    label18 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label18.grid(column=7, row=28,pady = 4, padx = 4)
    label19 = ttk.Label(mainframe, text="1000bf",font=("Arial", 10))
    label19.grid(column=4, row=29,pady = 4, padx = 4)
    label20 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label20.grid(column=7, row=29,pady = 4, padx = 4)
    label21 = ttk.Label(mainframe, text="1000lbf",font=("Arial", 10))
    label21.grid(column=4, row=30,pady = 4, padx = 4)
    label22 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label22.grid(column=7, row=30,pady = 4, padx = 4)
    label23 = ttk.Label(mainframe, text="1000lbf",font=("Arial", 10))
    label23.grid(column=4, row=31,pady = 4, padx = 4)
    label24 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label24.grid(column=7, row=31,pady = 4, padx = 4)
    label25 = ttk.Label(mainframe, text="1000lbf",font=("Arial", 10))
    label25.grid(column=4, row=32,pady = 4, padx = 4)
    label26 = ttk.Label(mainframe, text="gpm",font=("Arial", 10))
    label26.grid(column=7, row=32,pady = 4, padx = 4)
    print(int(unit.get()))
    
ttk.Button(mainframe, text="Field Units", command=field_unit).grid(column=4, row=1, pady = 4, padx = 4)
    
def SI_unit():
    a = 1
    unit.set(a)
    global label27
    global label28
    global label29
    global label30
    global label31
    global label32
    global label33
    global label34
    global label35
    global label36
    global label37
    global label38
    global label39
    global label40
    global label41
    global label42
    global label43
    global label44
    global label45
    global label46
    global label47
    global label48
    global label49
    global label50
    global label51
    global label52
    global label58
    global label59
    global label60
    global label61
    global label62
    global label64
    
    label27 = ttk.Label(mainframe, text="Pa",font=("Arial", 10))
    label27.grid(column=3, row=2,pady = 4, padx = 4)
    label28 = ttk.Label(mainframe, text="Pa",font=("Arial", 10))
    label28.grid(column=3, row=3,pady = 4,padx = 4)
    label29 = ttk.Label(mainframe, text="Pa",font=("Arial", 10))
    label29.grid(column=3, row=4,pady = 4,padx = 4)
    #ttk.Label(mainframe, text="inch",font=("Arial", 10)).grid(column=3, row=5,pady = 2,padx = 2, sticky=W)
    label30 = ttk.Label(mainframe, text="Pa",font=("Arial", 10))
    label30.grid(column=3, row=5,pady = 4,padx = 4)
    label31 = ttk.Label(mainframe, text="kg/m3",font=("Arial", 10))
    label31.grid(column=3, row=6,pady = 4,padx = 4)
    label58 = ttk.Label(mainframe, text="Pa*s",font=("Arial", 10))
    label58.grid(column=3, row=7,pady = 4,padx = 4)
    label59 = ttk.Label(mainframe, text="degree",font=("Arial", 10))
    label59.grid(column=3, row=8,pady = 4,padx = 4)
    label60 = ttk.Label(mainframe, text="mm",font=("Arial", 10))
    label60.grid(column=3, row=9,pady = 4,padx = 4)

    label32 = ttk.Label(mainframe, text="m",font=("Arial", 10))
    label32.grid(column=6, row=2,pady = 4, padx = 4)
    label33 = ttk.Label(mainframe, text="m",font=("Arial", 10))
    label33.grid(column=6, row=3,pady = 4,padx = 4)
    label34 = ttk.Label(mainframe, text="m",font=("Arial", 10))
    label34.grid(column=6, row=4,pady = 4,padx = 4)
    label35 = ttk.Label(mainframe, text="m",font=("Arial", 10))
    label35.grid(column=6, row=5,pady = 4,padx = 4)
    label36 = ttk.Label(mainframe, text="mm",font=("Arial", 10))
    label36.grid(column=6, row=6,pady = 4,padx = 4)
    label37 = ttk.Label(mainframe, text="mm",font=("Arial", 10))
    label37.grid(column=6, row=7,pady = 4,padx = 4)
    label38 = ttk.Label(mainframe, text="mm",font=("Arial", 10))
    label38.grid(column=6, row=8,pady = 4,padx = 4)
    label61 = ttk.Label(mainframe, text="mm",font=("Arial", 10))
    label61.grid(column=6, row=9,pady = 4,padx = 4)
    #label57 = ttk.Label(mainframe, text="inch",font=("Arial", 10))
    #label57.grid(column=6, row=10,pady = 4,padx = 4)
    label62 = ttk.Label(mainframe, text="m",font=("Arial", 10))
    label62.grid(column=6, row=12,pady = 4,padx = 4)
    label64 = ttk.Label(mainframe, text="m3/s",font=("Arial", 10))
    label64.grid(column=12, row=29,pady = 4, padx = 4)

    label39 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label39.grid(column=3, row=21,pady = 4, padx = 4)
    label40 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label40.grid(column=3, row=22,pady = 4,padx = 4)
    label41 = ttk.Label(mainframe, text="m3/s",font=("Arial", 10))
    label41.grid(column=3, row=23,pady = 4,padx = 4)
    label42 = ttk.Label(mainframe, text="m3/s",font=("Arial", 10))
    label42.grid(column=3, row=24,pady = 4,padx = 4)

    label43 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label43.grid(column=4, row=28,pady = 4, padx = 4)
    label44 = ttk.Label(mainframe, text="m3/sec",font=("Arial", 10))
    label44.grid(column=7, row=28,pady = 4, padx = 4)
    label45 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label45.grid(column=4, row=29,pady = 4, padx = 4)
    label46 = ttk.Label(mainframe, text="m3/sec",font=("Arial", 10))
    label46.grid(column=7, row=29,pady = 4, padx = 4)
    label47 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label47.grid(column=4, row=30,pady = 4, padx = 4)
    label48 = ttk.Label(mainframe, text="m3/sec",font=("Arial", 10))
    label48.grid(column=7, row=30,pady = 4, padx = 4)
    label49 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label49.grid(column=4, row=31,pady = 4, padx = 4)
    label50 = ttk.Label(mainframe, text="m3/sec",font=("Arial", 10))
    label50.grid(column=7, row=31,pady = 4, padx = 4)
    label51 = ttk.Label(mainframe, text="kN",font=("Arial", 10))
    label51.grid(column=4, row=32,pady = 4, padx = 4)
    label52 = ttk.Label(mainframe, text="m3/sec",font=("Arial", 10))
    label52.grid(column=7, row=32,pady = 4, padx = 4)
    print(int(unit.get()))
    
ttk.Button(mainframe, text="SI Units", command=SI_unit).grid(column=6, row=1, pady = 4, padx = 4)

def clear_field_units():
    label1.destroy()
    label2.destroy()
    label3.destroy()
    label4.destroy()
    label5.destroy()
    label6.destroy()
    label7.destroy()
    label8.destroy()
    label9.destroy()
    label10.destroy()
    label11.destroy()
    label12.destroy()
    label13.destroy()
    label14.destroy()
    label15.destroy()
    label16.destroy()
    label17.destroy()
    label18.destroy()
    label19.destroy()
    label20.destroy()
    label21.destroy()
    label22.destroy()
    label23.destroy()
    label24.destroy()
    label25.destroy()
    label26.destroy()
    label53.destroy()
    label54.destroy()
    label55.destroy()
    label56.destroy()
    label57.destroy()
    label63.destroy()
    
    
def clear_SI_units():
    label27.destroy()
    label28.destroy()
    label29.destroy()
    label30.destroy()
    label31.destroy()
    label32.destroy()
    label33.destroy()
    label34.destroy()
    label35.destroy()
    label36.destroy()
    label37.destroy()
    label38.destroy()
    label39.destroy()
    label40.destroy()
    label41.destroy()
    label42.destroy()
    label43.destroy()
    label44.destroy()
    label45.destroy()
    label46.destroy()
    label47.destroy()
    label48.destroy()
    label49.destroy()
    label50.destroy()
    label51.destroy()
    label52.destroy()
    label58.destroy()
    label59.destroy()
    label60.destroy()
    label61.destroy()
    label62.destroy()
    label64.destroy()
    
ttk.Button(mainframe, text="Clear Field Units", command=clear_field_units).grid(column=5, row=1, pady = 4, padx = 4)

ttk.Button(mainframe, text="Clear SI Units", command=clear_SI_units).grid(column=7, row=1, pady = 4, padx = 4)
    

#Optimize 
ttk.Button(mainframe, text="Optimize NN", command=best_nn).grid(column=10, row=2, pady = 4, padx = 4)
#ttk.Button(mainframe, text="Train NN", command=train_nn).grid(column=22, row=2, pady = 4, padx = 40, sticky=E)
ttk.Button(mainframe, text="Train NN", command=nn_performance).grid(column=12, row=2, pady = 4, padx = 4)

ttk.Label(mainframe, text="RMSE for test set ANN",font=("Arial", 10),justify=RIGHT).grid(column=10, row=3,pady = 4, padx = 4)
ttk.Label(mainframe, text="R^2 for test set ANN",font=("Arial", 10),justify = LEFT).grid(column=12, row=3,pady = 4,padx = 4)
ttk.Label(mainframe,background="white", textvariable=rmse_nn).grid(column=10, row=4,pady = 4, padx = 4)
ttk.Label(mainframe,background="white", textvariable=R_square_nn).grid(column=12, row=4,pady = 4, padx = 4)

ttk.Button(mainframe, text="Optimize Boosting", command=best_cat).grid(column=10, row=5, pady = 4, padx = 4)
#ttk.Button(mainframe, text="Train Boosting", command=best_nn).grid(column=22, row=5, pady = 4, padx = 40, sticky=E)
ttk.Button(mainframe, text="Train Boosting", command=cat_performance).grid(column=12, row=5, pady = 4, padx = 4)

ttk.Label(mainframe, text="RMSE for test set Boosting",font=("Arial", 10),justify=RIGHT).grid(column=10, row=6,pady = 4, padx = 4)
ttk.Label(mainframe, text="R^2 for test set Boosting",font=("Arial", 10),justify = LEFT).grid(column=12, row=6,pady = 4,padx = 4)
ttk.Label(mainframe,background="white", textvariable=rmse_cat).grid(column=10, row=7,pady = 4, padx = 4)
ttk.Label(mainframe,background="white", textvariable=R_square_cat).grid(column=12, row=7,pady = 4, padx = 4)

ttk.Button(mainframe, text="Train for Annular Loss", command=cat_performance_annuli).grid(column=12, row=8, pady = 4, padx = 4)

#Define Limits
#ttk.Button(mainframe, text="Set limits", command=define_limits).grid(column=1, row = 10, pady = 8,padx = 4)
ttk.Label(mainframe, text="Define Limits",font=("Arial", 10)).grid(column=2, row=20,pady = 8, padx = 4)

ttk.Label(mainframe, text="Min WOB",font=("Arial", 10)).grid(column=1, row=21,pady = 8, padx = 4)
ttk.Label(mainframe, text="Max WOB",font=("Arial", 10)).grid(column=1, row=22,pady = 8,padx = 4)
ttk.Label(mainframe, text="Min Flow Rate",font=("Arial", 10)).grid(column=1, row=23,pady = 8,padx = 4)
ttk.Label(mainframe, text="Max Flow Rate",font=("Arial", 10)).grid(column=1, row=24,pady = 8,padx = 4)

#ttk.Label(mainframe, background="white", textvariable=min_wob).grid(column=2, row=11,pady = 4, padx = 4)
#ttk.Label(mainframe, background="white", textvariable=max_wob).grid(column=2, row=12,pady = 4,padx = 4)
#ttk.Label(mainframe, background="white", textvariable=min_flow).grid(column=2, row=13,pady = 4,padx = 4)
#ttk.Label(mainframe, background="white", textvariable=max_flow).grid(column=2, row=14,pady = 4,padx = 4)


#Select The range
ttk.Label(mainframe, text="Define the Range",font=("Arial", 10)).grid(column=10, row=20,pady = 4, padx = 4)
ttk.Label(mainframe, text="Lower Boundary",font=("Arial", 10)).grid(column=8, row=21,pady = 4, padx = 4)
ttk.Label(mainframe, text="Upper Boundary",font=("Arial", 10)).grid(column=11, row=21,pady = 4, padx = 4)
#ttk.Label(mainframe, text="1000lbf",font=("Arial", 10)).grid(column=3, row=12,pady = 1,padx = 10, sticky=W)

#Define the number of combinations
ttk.Label(mainframe, text="Define Number of Combinations",font=("Arial", 10)).grid(column=1, row=25,pady = 8,padx = 4,sticky = S)

#Generate solutions
ttk.Button(mainframe, text="Generate Solutions", command=genetic_algorithm).grid(column=1, row=27, pady = 10, padx = 4)
ttk.Label(mainframe, text="Combination 1",font=("Arial", 10)).grid(column=1, row=28,pady = 4, padx = 4)
ttk.Label(mainframe, text="Combination 2",font=("Arial", 10)).grid(column=1, row=29,pady = 4,padx = 4)
ttk.Label(mainframe, text="Combination 3",font=("Arial", 10)).grid(column=1, row=30,pady = 4,padx = 4)
ttk.Label(mainframe, text="Combination 4",font=("Arial", 10)).grid(column=1, row=31,pady = 4,padx = 4)
ttk.Label(mainframe, text="Combination 5",font=("Arial", 10)).grid(column=1, row=32,pady = 4,padx = 4)

ttk.Label(mainframe, text="WOB",font=("Arial", 10)).grid(column=2, row=28,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=wob1).grid(column=3, row=28,pady = 4,padx = 4)


ttk.Label(mainframe, text="Flow Rate",font=("Arial", 10)).grid(column=5, row=28,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=flow1).grid(column=6, row=28,pady = 4,padx = 4)


ttk.Label(mainframe, text="WOB",font=("Arial", 10)).grid(column=2, row=29,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=wob2).grid(column=3, row=29,pady = 4,padx = 4)


ttk.Label(mainframe, text="Flow Rate",font=("Arial", 10)).grid(column=5, row=29,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=flow2).grid(column=6, row=29,pady = 4,padx = 4)


ttk.Label(mainframe, text="WOB",font=("Arial", 10)).grid(column=2, row=30,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=wob3).grid(column=3, row=30,pady = 4,padx = 4)


ttk.Label(mainframe, text="Flow Rate",font=("Arial", 10)).grid(column=5, row=30,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=flow3).grid(column=6, row=30,pady = 4,padx = 4)


ttk.Label(mainframe, text="WOB",font=("Arial", 10)).grid(column=2, row=31,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=wob4).grid(column=3, row=31,pady = 4,padx = 4)


ttk.Label(mainframe, text="Flow Rate",font=("Arial", 10)).grid(column=5, row=31,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=flow4).grid(column=6, row=31,pady = 4,padx = 4)


ttk.Label(mainframe, text="WOB",font=("Arial", 10)).grid(column=2, row=32,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=wob5).grid(column=3, row=32,pady = 4,padx = 4)


ttk.Label(mainframe, text="Flow Rate",font=("Arial", 10)).grid(column=5, row=32,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=flow5).grid(column=6, row=32,pady = 4,padx = 4)

ttk.Label(mainframe, text="Ratio",font=("Arial", 10)).grid(column=8, row=28,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=best_ratio1).grid(column=9, row=28,pady = 4,padx = 4)

ttk.Label(mainframe, text="Ratio",font=("Arial", 10)).grid(column=8, row=29,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=best_ratio2).grid(column=9, row=29,pady = 4,padx = 4)

ttk.Label(mainframe, text="Ratio",font=("Arial", 10)).grid(column=8, row=30,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=best_ratio3).grid(column=9, row=30,pady = 4,padx = 4)

ttk.Label(mainframe, text="Ratio",font=("Arial", 10)).grid(column=8, row=31,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=best_ratio4).grid(column=9, row=31,pady = 4,padx = 4)

ttk.Label(mainframe, text="Ratio",font=("Arial", 10)).grid(column=8, row=32,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=best_ratio5).grid(column=9, row=32,pady = 4,padx = 4)

ttk.Label(mainframe, text="Optimize Hole Cleaning",font=("Arial", 10)).grid(column=4, row=20,pady = 4, padx = 4)
ttk.Label(mainframe, text="ROP",font=("Arial", 10)).grid(column=10, row=28,pady = 4, padx = 4)
ttk.Label(mainframe, text="Initial ESV",font=("Arial", 10)).grid(column=12, row=28,pady = 4, padx = 4)
ttk.Label(mainframe, background="white", textvariable=Q_hole).grid(column=11, row=29,pady = 4,padx = 4)

#ttk.Label(mainframe, text="gpm",font=("Arial", 10)).grid(column=12, row=29,pady = 4, padx = 4)

#Insert variables
tau_entry = ttk.Entry(mainframe, width=10, textvariable=t)
tau_entry.grid(column=2, row=2,pady = 4,padx = 4)
theta600_entry = ttk.Entry(mainframe, width=10,textvariable=th600)
theta600_entry.grid(column=2, row=3,pady = 4,padx = 4)
theta300_entry = ttk.Entry(mainframe, width=10,textvariable=th300)
theta300_entry.grid(column=2, row=4,pady = 4,padx = 4)
#diameter_entry = ttk.Entry(mainframe, width=10,textvariable = Diameter)
#diameter_entry.grid(column=2, row=5,pady = 4,padx = 4, sticky=(E))
yield_entry = ttk.Entry(mainframe, width=10,textvariable = Yield)
yield_entry.grid(column=2, row=5,pady = 4,padx = 4)
density_entry = ttk.Entry(mainframe, width=10,textvariable = ro)
density_entry.grid(column=2, row=6,pady = 4,padx = 4)
viscosity_entry = ttk.Entry(mainframe, width=10,textvariable = visc_p)
viscosity_entry.grid(column=2, row=7,pady = 4,padx = 4)
well_angle_entry = ttk.Entry(mainframe, width=10,textvariable = angle)
well_angle_entry.grid(column=2, row=8,pady = 4,padx = 4)
av_size_entry = ttk.Entry(mainframe, width=10,textvariable = D50)
av_size_entry.grid(column=2, row=9,pady = 4,padx = 4)
dc_length_entry = ttk.Entry(mainframe, width=10,textvariable = length_dc)
dc_length_entry.grid(column=5, row=2,pady = 4,padx = 4)
lwd_length_entry = ttk.Entry(mainframe, width=10,textvariable = length_lwd)
lwd_length_entry.grid(column=5, row=3,pady = 4,padx = 4)
mwd_length_entry = ttk.Entry(mainframe, width=10,textvariable = length_mwd)
mwd_length_entry.grid(column=5, row=4,pady = 4,padx = 4)
motor_length_entry = ttk.Entry(mainframe, width=10,textvariable = length_motor)
motor_length_entry.grid(column=5, row=5,pady = 4,padx = 4)
dc_diam_entry = ttk.Entry(mainframe, width=10,textvariable = D_dc)
dc_diam_entry.grid(column=5, row=6,pady = 4,padx = 4)
dp_diam = ttk.Entry(mainframe, width=10,textvariable = D_dp)
dp_diam.grid(column=5, row=7,pady = 4,padx = 4)
dс_diam_ex = ttk.Entry(mainframe, width=10,textvariable = D_dc_ex)
dс_diam_ex.grid(column=5, row=8,pady = 4,padx = 4)
Dhole = ttk.Entry(mainframe, width=10,textvariable = D_hole)
Dhole.grid(column=5, row=9,pady = 4,padx = 4)

crit_low_entry = ttk.Entry(mainframe, width=10,textvariable = criterion_lower)
crit_low_entry.grid(column=9, row=21,pady = 4,padx = 4)
crit_up_entry = ttk.Entry(mainframe, width=10,textvariable = criterion_upper)
crit_up_entry.grid(column=12, row=21,pady = 4,padx = 4)
combination_entry = ttk.Entry(mainframe, width=10,textvariable = num_combinations)
combination_entry.grid(column=2, row=25,pady = 4,padx = 4)
min_wob_entry = ttk.Entry(mainframe, width=10,textvariable = min_wob)
min_wob_entry.grid(column=2, row=21,pady = 4,padx = 4)
max_wob_entry = ttk.Entry(mainframe, width=10,textvariable = max_wob)
max_wob_entry.grid(column=2, row=22,pady = 4,padx = 4)
min_flow_entry = ttk.Entry(mainframe, width=10,textvariable = min_flow)
min_flow_entry.grid(column=2, row=23,pady = 4,padx = 4)
max_flow_entry = ttk.Entry(mainframe, width=10,textvariable = max_flow)
max_flow_entry.grid(column=2, row=24,pady = 4,padx = 4)
rop_entry = ttk.Entry(mainframe, width=10,textvariable = ROP)
rop_entry.grid(column=11, row=28,pady = 4,padx = 4)
esv_entry = ttk.Entry(mainframe, width=10,textvariable = ESV)
esv_entry.grid(column=13, row=28,pady = 4,padx = 4)

text = ttk.Entry(mainframe,textvariable=var)
text.grid(row = 1,column = 2,pady = 4, padx = 4, sticky=E)
ttk.Button(mainframe, text="Browse File", command=browsefunc).grid(column=1, row=1, pady = 4, padx = 4)
ttk.Button(mainframe, text="Plot Inputs", command=plot_inputs).grid(column=1, row=18, pady = 4, padx = 4)
ttk.Button(mainframe, text="Optimize Flow Rate", command=hole_cleaning_effect).grid(column=10, row=29, pady = 4, padx = 4)

ttk.Button(mainframe, text="Exit Program", command=close_window).grid(column=10, row=20, pady = 4, padx = 4)
#ttk.Button(mainframe, text="Clear Graphs", command=delete_graphs).grid(column=2, row=8, pady = 1, padx = 1, sticky=E)




# this will create a label widget

#l1 = Label(master, text = "Height")
#l2 = Label(master, text = "Width")

  
# grid method to arrange labels in respective
# rows and columns as specified
#l1.grid(row = 0, column = 0, sticky = W, pady = 4)
#l2.grid(row = 1, column = 0, sticky = W, pady = 4)
  
# entry widgets, used to take entry from user
#e1 = Entry(master)
#e2 = Entry(master)
  
# this will arrange entry widgets
#e1.grid(row = 0, column = 1, pady = 4)
#e2.grid(row = 1, column = 1, pady = 4)

root.mainloop()